In [ ]:
# https://github.com/Tokemak/v2-swap-routes-watcher/blob/main/src/core/helpers/aggregator/coingecko.ts
# based on this

from mainnet_launch.constants import ChainData, eth_client, COINGECKO_API_KEY, ETH_CHAIN, BASE_CHAIN, SONIC_CHAIN

import pandas as pd
import requests
from pprint import pprint

base_url = "https://pro-api.coingecko.com/api/v3"

GHO = eth_client.toChecksumAddress("0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f").lower()


def fetch_coingeck_pools(start_token: str, chain: ChainData, min_USD_reserves: int = 100_000):

    if chain == ETH_CHAIN:
        coingecko_coin_slug = "eth"
    elif chain == BASE_CHAIN:
        coingecko_coin_slug = "base"
    elif chain == SONIC_CHAIN:
        coingecko_coin_slug = "sonic"

    url = f"{base_url}/onchain/networks/{coingecko_coin_slug}/tokens/{start_token}/pools"
    headers = {"x-cg-pro-api-key": COINGECKO_API_KEY}
    params = {"sort": "h24_volume_usd_liquidity_desc", "include": ["base_token", "quote_token"]}

    resp = requests.get(url, headers=headers, params=params, timeout=30)
    resp.raise_for_status()
    pools = resp.json()["data"]

    def _extract_pool_fields_from_coingecko(pools):
        rows = []
        for p in pools:
            row = {
                "address": p["attributes"]["address"],
                "name": p["attributes"]["name"],
                "reserve_in_usd": round(float(p["attributes"]["reserve_in_usd"])),
                "base_token_id": p["relationships"]["base_token"]["data"]["id"].split("_")[1],
                "quote_token_id": p["relationships"]["quote_token"]["data"]["id"].split("_")[1],
                "dex_id": p["relationships"]["dex"]["data"]["id"],
            }

            rows.append(row)

        return pd.DataFrame.from_records(rows)

    df = _extract_pool_fields_from_coingecko(pools)

    return df[df["reserve_in_usd"] >= min_USD_reserves]


def fetch_n_hops_from_tokens_with_coingecko(
    tokens_to_check: set[str], chain: str, min_USD_reserves: int = 100_000, n_hops: int = 2
) -> pd.DataFrame:
    tokens_to_check = set([eth_client.toChecksumAddress(t).lower() for t in tokens_to_check])
    tokens_already_checked = set()

    dex_liqudity_df = None

    for hop_num in range(n_hops):
        tokens_to_check_this_hop = [t for t in tokens_to_check if t not in tokens_already_checked]
        print(f"Checking {len(tokens_to_check_this_hop)} tokens in this hop. {hop_num}")
        for token in tokens_to_check_this_hop:
            if dex_liqudity_df is None:
                dex_liqudity_df = fetch_coingeck_pools(token, chain, min_USD_reserves=min_USD_reserves)
            else:
                this_token_hops_df = fetch_coingeck_pools(token, chain, min_USD_reserves=min_USD_reserves)
                dex_liqudity_df = pd.concat([dex_liqudity_df, this_token_hops_df], ignore_index=True)

            tokens_already_checked.add(eth_client.toChecksumAddress(token))

        new_tokens_to_check = set(
            dex_liqudity_df["quote_token_id"].tolist() + dex_liqudity_df["base_token_id"].tolist()
        )
        new_tokens_to_check = set([eth_client.toChecksumAddress(t).lower() for t in new_tokens_to_check])
        tokens_to_check = new_tokens_to_check

    return dex_liqudity_df


dex_liqudity_df

Skipping 0xc81b27ac184a631a0cbde20b51c734664b04d5b3 as it has already been searched.
Skipping 0x4c9edd5852cd905f086c759e8383e09bff1e68b3 as it has already been searched.
Skipping 0xc2b021133d1b0cf07dba696fd5dd89338428225b as it has already been searched.
Skipping 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 as it has already been searched.
Skipping 0x085780639cc2cacd35e474e71f4d000e2405d8f6 as it has already been searched.
Skipping 0x66a1e37c9b0eaddca17d3662d6c05f4decf3e110 as it has already been searched.
Skipping 0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f as it has already been searched.
Skipping 0xcbb7c0000ab88b473b1f5afd9ef808440eed33bf as it has already been searched.
Skipping 0xf939e0a03fb07f59a73314e73794be0e57ac1b4e as it has already been searched.
Skipping 0x8353157092ed8be69a9df8f95af097bbf33cb2af as it has already been searched.


In [65]:
df = pd.concat(other_dfs, ignore_index=True)
df.drop_duplicates()

,address,name,reserve_in_usd,base_token_id,quote_token_id,dex_id
0,0x4628f13651ead6793f8d838b34b8f8522fb0cc52,GHO / USR,7690522,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0x66a1e37c9b0eaddca17d3662d6c05f4decf3e110,curve
1,0x74345504eaea3d9408fc69ae7eb2d14095643c5b,GHO / fxUSD,3951535,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0x085780639cc2cacd35e474e71f4d000e2405d8f6,curve
2,0x635ef0056a597d13863b73825cca297236578595,GHO / crvUSD,2675961,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0xf939e0a03fb07f59a73314e73794be0e57ac1b4e,curve
3,0x670a72e6d22b0956c0d2573288f82dcc5d6e3a61,GHO / USDe,1826167,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0x4c9edd5852cd905f086c759e8383e09bff1e68b3,curve
4,0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d,GHO / USDC 0.05%,897749,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,uniswap_v3
...,...,...,...,...,...,...
282,0xc30c8b862f7de6ba5d7eaeb113c78ec6b5ded04b,PYUSD / USDT 0.01%,1379485,0x6c3ea9036406852006290770bedfcaba0e23a0e8,0xdac17f958d2ee523a2206206994597c13d831ec7,uniswap_v3
284,0x13394005c1012e708fce1eb974f1130fdc73a5ce,PYUSD / USDC 0.01%,500353,0x6c3ea9036406852006290770bedfcaba0e23a0e8,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,uniswap_v3
285,0xaa4c9d6e5e349f319abb625aa8dca5f52abea757,PYUSD / USDC 0.05%,279628,0x6c3ea9036406852006290770bedfcaba0e23a0e8,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,uniswap_v3
286,0xdd2e0d86a45e4ef9bd490c2809e6405720cc357c,PYUSD / USDT 0.3%,454230,0x6c3ea9036406852006290770bedfcaba0e23a0e8,0xdac17f958d2ee523a2206206994597c13d831ec7,uniswap_v3


In [ ]:
# that can be shared,
# and the above can be shared too,
# dex screener

# totally to able, aldo need some tokens to exclude

In [ ]:
# need a whitelist of tokens to keep, and what to exclude

# pipe this to dex screener to get the TVL in USD for each side

# then do the path finding,

# max weight, where weight is the TVL in USD of the largest of the non start toekens in the pool

In [58]:
other_dfs = []
for token in to_search_tokens:
    print(token)
    if eth_client.toChecksumAddress(token) not in already_searched_tokens:
        other_df = fetch_coingeck_pools(token, min_USD_reserves=100_000)
        other_dfs.append(other_df)
        already_searched_tokens.add(eth_client.toChecksumAddress(token))
    else:
        print(f"Skipping {token} as it has already been searched.")

0xc81b27ac184a631a0cbde20b51c734664b04d5b3
Skipping 0xc81b27ac184a631a0cbde20b51c734664b04d5b3 as it has already been searched.
0x4c9edd5852cd905f086c759e8383e09bff1e68b3
Skipping 0x4c9edd5852cd905f086c759e8383e09bff1e68b3 as it has already been searched.
0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
Skipping 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 as it has already been searched.
0xcbb7c0000ab88b473b1f5afd9ef808440eed33bf
Skipping 0xcbb7c0000ab88b473b1f5afd9ef808440eed33bf as it has already been searched.
0xf939e0a03fb07f59a73314e73794be0e57ac1b4e
Skipping 0xf939e0a03fb07f59a73314e73794be0e57ac1b4e as it has already been searched.
0x085780639cc2cacd35e474e71f4d000e2405d8f6
Skipping 0x085780639cc2cacd35e474e71f4d000e2405d8f6 as it has already been searched.
0x66a1e37c9b0eaddca17d3662d6c05f4decf3e110
Skipping 0x66a1e37c9b0eaddca17d3662d6c05f4decf3e110 as it has already been searched.
0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f
Skipping 0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f as it has

In [56]:
dfs = pd.concat([start_df] + other_dfs, ignore_index=True)
dfs["address"].value_counts()

address
0x4628f13651ead6793f8d838b34b8f8522fb0cc52    10
0x74345504eaea3d9408fc69ae7eb2d14095643c5b    10
0x635ef0056a597d13863b73825cca297236578595    10
0x670a72e6d22b0956c0d2573288f82dcc5d6e3a61    10
0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d    10
0x8a4f252812dff2a8636e4f7eb249d8fc2e3bd77f    10
0x8353157092ed8be69a9df8f95af097bbf33cb2af    10
0xc2b021133d1b0cf07dba696fd5dd89338428225b    10
0xb6e07cce63bf54974a8379d9be77d997c28fac08    10
Name: count, dtype: int64

In [37]:
import pandas as pd
import requests
from pprint import pprint
from mainnet_launch.constants import COINGECKO_API_KEY, eth_client  # keep your existing import style

base_url = "https://pro-api.coingecko.com/api/v3"
POOL_ADDR = eth_client.toChecksumAddress("0x4628f13651ead6793f8d838b34b8f8522fb0cc52")  # ← any pool

url = f"{base_url}/onchain/networks/eth/pools/{POOL_ADDR}"
headers = {"x-cg-pro-api-key": COINGECKO_API_KEY}
params = {"include": "base_token,quote_token"}  # comma‑sep string (FG docs treat list as 400)

resp = requests.get(url, headers=headers, params=params, timeout=30)
resp.raise_for_status()

pool_data = resp.json()["data"]
pool_data

{'id': 'eth_0x4628f13651ead6793f8d838b34b8f8522fb0cc52',
 'type': 'pool',
 'attributes': {'base_token_price_usd': '0.998048805541752',
  'base_token_price_native_currency': '0.00028760389990886799193937861081916001196708328',
  'quote_token_price_usd': '1.14434811101979',
  'quote_token_price_native_currency': '0.000329762410169899523278607173632',
  'base_token_price_quote_token': '0.8721548941',
  'quote_token_price_base_token': '1.1465853219',
  'address': '0x4628f13651ead6793f8d838b34b8f8522fb0cc52',
  'name': 'GHO / USR',
  'pool_name': 'GHO / USR',
  'pool_fee_percentage': None,
  'pool_created_at': '2025-01-15T17:32:59Z',
  'fdv_usd': '311476802.434238',
  'market_cap_usd': '311476812.97369',
  'price_change_percentage': {'m5': '0',
   'm15': '0',
   'm30': '0',
   'h1': '0',
   'h6': '1.26',
   'h24': '1.4'},
  'transactions': {'m5': {'buys': 0, 'sells': 0, 'buyers': 0, 'sellers': 0},
   'm15': {'buys': 0, 'sells': 0, 'buyers': 0, 'sellers': 0},
   'm30': {'buys': 1, 'sells': 0

In [31]:
raw = resp.json()["data"]
pprint(raw[0])

{'attributes': {'address': '0x4628f13651ead6793f8d838b34b8f8522fb0cc52',
                'base_token_price_native_currency': '0.00028760389990886799193937861081916001196708328',
                'base_token_price_quote_token': '0.8721548941',
                'base_token_price_usd': '0.998048805541752',
                'fdv_usd': '311476802.434238',
                'market_cap_usd': '311476812.97369',
                'name': 'GHO / USR',
                'pool_created_at': '2025-01-15T17:32:59Z',
                'price_change_percentage': {'h1': '0',
                                            'h24': '1.4',
                                            'h6': '1.26',
                                            'm15': '0',
                                            'm30': '0',
                                            'm5': '0'},
                'quote_token_price_base_token': '1.1465853219',
                'quote_token_price_native_currency': '0.000329762410169899523278607173632',
      

In [28]:
pprint(pool)

{'attributes': {'address': '0x4628f13651ead6793f8d838b34b8f8522fb0cc52',
                'base_token_price_native_currency': '0.00028760389990886799193937861081916001196708328',
                'base_token_price_quote_token': '0.8721548941',
                'base_token_price_usd': '0.998048805541752',
                'fdv_usd': '311476802.434238',
                'market_cap_usd': '311476812.97369',
                'name': 'GHO / USR',
                'pool_created_at': '2025-01-15T17:32:59Z',
                'price_change_percentage': {'h1': '0',
                                            'h24': '1.4',
                                            'h6': '1.26',
                                            'm15': '0',
                                            'm30': '0',
                                            'm5': '0'},
                'quote_token_price_base_token': '1.1465853219',
                'quote_token_price_native_currency': '0.000329762410169899523278607173632',
      

In [52]:
def extract_pool_fields(pools):
    rows = []
    for p in pools:
        row = {
            "address": p["attributes"]["address"],
            "name": p["attributes"]["name"],
            "reserve_in_usd": round(float(p["attributes"]["reserve_in_usd"])),
            "base_token_id": p["relationships"]["base_token"]["data"]["id"].split("_")[1],
            "quote_token_id": p["relationships"]["quote_token"]["data"]["id"].split("_")[1],
            "dex_id": p["relationships"]["dex"]["data"]["id"],
        }

        rows.append(row)

    df = pd.DataFrame.from_records(rows)

    return df[df["reserve_in_usd"] >= 100000]


df = extract_pool_fields(raw)
df

,address,name,reserve_in_usd,base_token_id,quote_token_id,dex_id
0,0x4628f13651ead6793f8d838b34b8f8522fb0cc52,GHO / USR,7688047,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0x66a1e37c9b0eaddca17d3662d6c05f4decf3e110,curve
1,0x74345504eaea3d9408fc69ae7eb2d14095643c5b,GHO / fxUSD,3985657,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0x085780639cc2cacd35e474e71f4d000e2405d8f6,curve
2,0x635ef0056a597d13863b73825cca297236578595,GHO / crvUSD,2672583,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0xf939e0a03fb07f59a73314e73794be0e57ac1b4e,curve
3,0x670a72e6d22b0956c0d2573288f82dcc5d6e3a61,GHO / USDe,1827217,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0x4c9edd5852cd905f086c759e8383e09bff1e68b3,curve
4,0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d,GHO / USDC 0.05%,898885,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,uniswap_v3
5,0x8a4f252812dff2a8636e4f7eb249d8fc2e3bd77f,GHO / cbBTC / WETH,4550656,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0xcbb7c0000ab88b473b1f5afd9ef808440eed33bf,curve
6,0x8353157092ed8be69a9df8f95af097bbf33cb2af,GHO / GHO/USDT/USDC / USDC / USDT 0.05%,355793,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0x8353157092ed8be69a9df8f95af097bbf33cb2af,balancer_ethereum
12,0xc2b021133d1b0cf07dba696fd5dd89338428225b,GHO / GHO/bb-a-USD / bb-a-USD 0.05%,23209706,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0xc2b021133d1b0cf07dba696fd5dd89338428225b,balancer_ethereum
13,0xb6e07cce63bf54974a8379d9be77d997c28fac08,GHO / DMusd 0.01%,14006631,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0xc81b27ac184a631a0cbde20b51c734664b04d5b3,sushiswap-v3-ethereum


In [ ]:
# how about 2 hops?

# GHO all > 100k liquidty

# for all other tokens in (GHO / X)

# use X and put them into it.


# then do it again

# then track

In [ ]:
def extract_pool_fields(pools):
    """
    Strict extractor: index lookups only (no .get()).
    Raises KeyError (wrapped w/ pool index context) if schema mismatch.
    Returns list of dicts: address, name, base_token_id, quote_token_id.
    """
    rows = []
    for i, p in enumerate(pools):
        try:
            row = {
                "address": p["attributes"]["address"],
                "name": p["attributes"]["name"],
                "reserve_in_usd": round(float(p["attributes"]["reserve_in_usd"])),
                "base_token_id": p["relationships"]["base_token"]["data"]["id"],
                "quote_token_id": p["relationships"]["quote_token"]["data"]["id"],
                "dex_id": p["relationships"]["dex"]["data"]["id"],
            }
        except KeyError as e:
            raise KeyError(f"Missing key {e!r} in pool index {i}: {p}") from e

        rows.append(row)

    df = pd.DataFrame.from_records(rows)

    df = df[df["reserve_in_usd"] >= 100000]  # filter out pools with < 10k liquidity
    return df[df["reserve_in_usd"] >= 100000]


df = extract_pool_fields(raw)
df

In [ ]:
#

In [45]:
import pandas as pd, requests
from mainnet_launch.constants import COINGECKO_API_KEY, eth_client

BASE_URL = "https://pro-api.coingecko.com/api/v3"
POOL_ADDR = eth_client.toChecksumAddress("0x74345504eaea3d9408fc69ae7eb2d14095643c5b")

url = f"{BASE_URL}/onchain/networks/eth/pools/{POOL_ADDR}/ohlcv/day"
params = {
    "limit": 1,  # just the latest daily candle
    "include": "liquidity",  # appends USD split to each candle
}

resp = requests.get(url, params=params, headers={"x-cg-pro-api-key": COINGECKO_API_KEY}, timeout=30)
resp.raise_for_status()


# data = resp.json()
# latest = resp.json()["ohlcv_list"][-1]          # <-- 8‑element array
# (
#     ts, open_, high, low, close, vol,
#     usd_base, usd_quote,                 # added by include=liquidity
# ) = latest

# total = usd_base + usd_quote
# df = pd.DataFrame(
#     {
#         "token_side": ["base",  "quote"],
#         "reserve_usd": [usd_base, usd_quote],
#         "weight":      [usd_base/total, usd_quote/total],
#         "timestamp":   pd.to_datetime(ts, unit="s", utc=True),
#     }
# )

# print(df)

In [50]:
import requests, pandas as pd
from pprint import pprint

# --- replace these two lines as needed -------------------------------
CHAIN_ID = "ethereum"  # e.g. ethereum, polygon, bsc …
PAIR_ADDRESS = "0x4628f13651ead6793f8d838b34b8f8522fb0cc52".lower()
# ---------------------------------------------------------------------

url = f"https://api.dexscreener.com/latest/dex/pairs/{CHAIN_ID}/{PAIR_ADDRESS}"
resp = requests.get(url, timeout=30)
resp.raise_for_status()

pair = resp.json()["pairs"]  # DexScreener wraps result in a list

# # Pull the pieces we need
# liq  = pair["liquidity"]                 # {'usd', 'base', 'quote'}
# base = pair["baseToken"]                 # symbol / address / priceUsd …
# quote = pair["quoteToken"]

# usd_base  = float(base["priceUsd"])  * float(liq["base"])
# usd_quote = float(quote["priceUsd"]) * float(liq["quote"])
# total     = usd_base + usd_quote

# df = pd.DataFrame(
#     [
#         {
#             "token_symbol": base["symbol"],
#             "reserve_token_units": float(liq["base"]),
#             "price_usd": float(base["priceUsd"]),
#             "reserve_usd": usd_base,
#             "weight": usd_base / total,
#         },
#         {
#             "token_symbol": quote["symbol"],
#             "reserve_token_units": float(liq["quote"]),
#             "price_usd": float(quote["priceUsd"]),
#             "reserve_usd": usd_quote,
#             "weight": usd_quote / total,
#         },
#     ]
# )

# pprint(df)

In [51]:
pair

[{'chainId': 'ethereum',
  'dexId': 'curve',
  'url': 'https://dexscreener.com/ethereum/0x4628f13651ead6793f8d838b34b8f8522fb0cc52',
  'pairAddress': '0x4628f13651eaD6793F8d838B34B8f8522Fb0cc52',
  'labels': ['stbl'],
  'baseToken': {'address': '0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f',
   'name': 'Gho Token',
   'symbol': 'GHO'},
  'quoteToken': {'address': '0x66a1E37c9b0eAddca17d3662D6c05F4DECf3e110',
   'name': 'Resolv USD',
   'symbol': 'USR'},
  'priceNative': '0.9990',
  'priceUsd': '0.9992',
  'txns': {'m5': {'buys': 0, 'sells': 0},
   'h1': {'buys': 1, 'sells': 0},
   'h6': {'buys': 2, 'sells': 0},
   'h24': {'buys': 18, 'sells': 30}},
  'volume': {'h24': 5974829.27, 'h6': 205462.11, 'h1': 200971.21, 'm5': 0},
  'priceChange': {'h1': -0.05, 'h6': -0.02},
  'liquidity': {'usd': 7324100.08, 'base': 4895866, 'quote': 2431232},
  'fdv': 301781150,
  'marketCap': 301781150,
  'pairCreatedAt': 1736947163000}]

In [46]:
resp.json()

{'data': {'id': '0aded8ac-9f31-4a0e-a78f-5c6a14be73c3',
  'type': 'ohlcv_request_response',
  'attributes': {'ohlcv_list': [[1752710400,
     0.985722931408556,
     0.990480146693653,
     0.968773925281548,
     0.990480146693653,
     788074.802008096]]}},
 'meta': {'base': {'address': '0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f',
   'name': 'Gho Token',
   'symbol': 'GHO',
   'coingecko_coin_id': 'gho'},
  'quote': {'address': '0x085780639cc2cacd35e474e71f4d000e2405d8f6',
   'name': 'f(x) USD',
   'symbol': 'fxUSD',
   'coingecko_coin_id': 'f-x-protocol-fxusd'}}}

In [ ]:
# we can get the liquidity of the pool, but we don't know how much of that is in each token

# here are some optoins


# get (safe price and liquidity) on chain, I don't trus

In [ ]:
# we can

In [ ]:
a cruve

In [27]:
pprint(pool)

{'attributes': {'address': '0x4628f13651ead6793f8d838b34b8f8522fb0cc52',
                'base_token_price_native_currency': '0.00028760389990886799193937861081916001196708328',
                'base_token_price_quote_token': '0.8721548941',
                'base_token_price_usd': '0.998048805541752',
                'fdv_usd': '311476802.434238',
                'market_cap_usd': '311476812.97369',
                'name': 'GHO / USR',
                'pool_created_at': '2025-01-15T17:32:59Z',
                'price_change_percentage': {'h1': '0',
                                            'h24': '1.4',
                                            'h6': '1.26',
                                            'm15': '0',
                                            'm30': '0',
                                            'm5': '0'},
                'quote_token_price_base_token': '1.1465853219',
                'quote_token_price_native_currency': '0.000329762410169899523278607173632',
      

In [ ]:
# other URLs to check
url = "https://pro-api.coingecko.com/api/v3/onchain/networks/network/tokens/token_address/pools"

In [22]:
df

,address,name,reserve_in_usd,base_token_id,quote_token_id
0,0x4628f13651ead6793f8d838b34b8f8522fb0cc52,GHO / USR,7.665772e+06,eth_0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,eth_0x66a1e37c9b0eaddca17d3662d6c05f4decf3e110
1,0x74345504eaea3d9408fc69ae7eb2d14095643c5b,GHO / fxUSD,3.985657e+06,eth_0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,eth_0x085780639cc2cacd35e474e71f4d000e2405d8f6
2,0x635ef0056a597d13863b73825cca297236578595,GHO / crvUSD,2.669743e+06,eth_0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,eth_0xf939e0a03fb07f59a73314e73794be0e57ac1b4e
3,0x670a72e6d22b0956c0d2573288f82dcc5d6e3a61,GHO / USDe,1.821923e+06,eth_0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,eth_0x4c9edd5852cd905f086c759e8383e09bff1e68b3
4,0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d,GHO / USDC 0.05%,8.953718e+05,eth_0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,eth_0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
5,0x8a4f252812dff2a8636e4f7eb249d8fc2e3bd77f,GHO / cbBTC / WETH,4.545821e+06,eth_0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,eth_0xcbb7c0000ab88b473b1f5afd9ef808440eed33bf
6,0x8353157092ed8be69a9df8f95af097bbf33cb2af,GHO / GHO/USDT/USDC / USDC / USDT 0.05%,3.557928e+05,eth_0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,eth_0x8353157092ed8be69a9df8f95af097bbf33cb2af
7,0x14cf6d2fe3e1b326114b07d22a6f6bb59e346c67,GHO / USDC 0.01%,1.095050e+04,eth_0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,eth_0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
8,0x2b3fec4acbfce6ae91d529f88a5d6d438de36a457907...,GHO / ETH 1%,1.075833e+04,eth_0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,eth_0x0000000000000000000000000000000000000000
9,0x8cd52ee292313c4d851e71a7064f096504ab3ee9,GHO / WBTC / wstETH,4.713358e+03,eth_0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,eth_0x2260fac5e5542a773aa44fbcfedf7c193bc2c599


In [ ]:
# for simplicty, exclude all pools with < 10k

In [21]:
from pprint import pprint

pprint(pool)

{'attributes': {'address': '0x4628f13651ead6793f8d838b34b8f8522fb0cc52',
                'base_token_price_native_currency': '0.00028760389990886799193937861081916001196708328',
                'base_token_price_quote_token': '0.8721548941',
                'base_token_price_usd': '0.998048805541752',
                'fdv_usd': '311476802.434238',
                'market_cap_usd': '311476812.97369',
                'name': 'GHO / USR',
                'pool_created_at': '2025-01-15T17:32:59Z',
                'price_change_percentage': {'h1': '0',
                                            'h24': '1.4',
                                            'h6': '1.26',
                                            'm15': '0',
                                            'm30': '0',
                                            'm5': '0'},
                'quote_token_price_base_token': '1.1465853219',
                'quote_token_price_native_currency': '0.000329762410169899523278607173632',
      

In [ ]:
# {'attributes': {'address': '0x4628f13651ead6793f8d838b34b8f8522fb0cc52',
#                 'base_token_price_native_currency': '0.00028760389990886799193937861081916001196708328',
#                 'base_token_price_quote_token': '0.8721548941',
#                 'base_token_price_usd': '0.998048805541752',
#                 'fdv_usd': '311476802.434238',
#                 'market_cap_usd': '311476812.97369',
#                 'name': 'GHO / USR',
#                 'pool_created_at': '2025-01-15T17:32:59Z',
#                 'price_change_percentage': {'h1': '0',
#                                             'h24': '1.4',
#                                             'h6': '1.26',
#                                             'm15': '0',
#                                             'm30': '0',
#                                             'm5': '0'},
#                 'quote_token_price_base_token': '1.1465853219',
#                 'quote_token_price_native_currency': '0.000329762410169899523278607173632',
#                 'quote_token_price_usd': '1.14434811101979',
#                 'reserve_in_usd': '7691692.9676',
#                 'token_price_usd': '0.998048805541752',
#                 'transactions': {'h1': {'buyers': 1,
#                                         'buys': 1,
#                                         'sellers': 0,
#                                         'sells': 0},
#                                  'h24': {'buyers': 7,
#                                          'buys': 18,
#                                          'sellers': 6,
#                                          'sells': 30},
#                                  'h6': {'buyers': 2,
#                                         'buys': 2,
#                                         'sellers': 0,
#                                         'sells': 0},
#                                  'm15': {'buyers': 0,
#                                          'buys': 0,
#                                          'sellers': 0,
#                                          'sells': 0},
#                                  'm30': {'buyers': 1,
#                                          'buys': 1,
#                                          'sellers': 0,
#                                          'sells': 0},
#                                  'm5': {'buyers': 0,
#                                         'buys': 0,
#                                         'sellers': 0,
#                                         'sells': 0}},
#                 'volume_usd': {'h1': '200721.878317723',
#                                'h24': '6003864.92574268',
#                                'h6': '205149.504992591',
#                                'm15': '0.0',
#                                'm30': '200721.878317723',
#                                'm5': '0.0'}},
#  'id': 'eth_0x4628f13651ead6793f8d838b34b8f8522fb0cc52',
#  'relationships': {'base_token': {'data': {'id': 'eth_0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f',
#                                            'type': 'token'}},
#                    'dex': {'data': {'id': 'curve', 'type': 'dex'}},
#                    'quote_token': {'data': {'id': 'eth_0x66a1e37c9b0eaddca17d3662d6c05f4decf3e110',
#                                             'type': 'token'}}},
#  'type': 'pool'}

# this is a GHO, USR pool, it is claiming that GHO/ USD is at a 1.14 price, when it should be lcsoe to 1
# this makes me not trust this values